In [50]:
# Importing libraries
import os
from dotenv import load_dotenv
import requests
import gradio as gr
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import display, Markdown

In [51]:
# Load environment variables from the .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "'your-key-if-not-using-env'")
openai = OpenAI()

In [52]:
# Website class to handle the extraction of title and text content from a webpage
class Website():
    url: str
    title: str
    text: str
    
    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content,'html.parser')
        self.title = soup.title.string if soup.title else 'no title exist'
        for irrelevant in soup.body(['script', 'input','img','style']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [54]:
# System prompt for summarizing website content
system_prompt = """
You are an assistant that analyzes website content and provides summaries. 
Respond in Markdown format and offer styles such as key points, detailed summaries or Q&A format.
"""

In [56]:
# Function to generate user-specific prompt based on style
def user_prompt(website, style):
    if style == "Key Points":
        user_prompt = f"Summarize the website titled '{website.title}' in key points:\n\n{website.text}"
    elif style == "Detailed":
        user_prompt = f"Provide a detailed summary of the website titled '{website.title}':\n\n{website.text}"
    elif style == "Q&A":
        user_prompt = f"Generate a question-answer format summary for the website titled '{website.title}':\n\n{website.text}"
    return user_prompt

In [57]:
# Define the function to structure the message for GPT
def message_for(website, style):
       return[{"role": "system", "content": system_prompt},
              {"role": "user","content": user_prompt(website, style)}]

In [58]:
# Define the summarize function to generate a concise summary for a given URL
def summarize(url, style):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages= message_for(website, style)
    )
    result = response.choices[0].message.content
    return f"Mock summary for '{website.title}' in {style} style.\n\n{result}"

In [72]:
# Function to display this nicely in the Jupyter output

def display_summary(url, style):
    summary = summarize(url, style)
    display(Markdown(summary))

In [74]:
display_summary('https://towardsdatascience.com','Key Points')

Mock summary for 'Towards Data Science' in Key Points style.

# Towards Data Science - Key Points Summary

- **Overview**: 
  - A leading publication focusing on data science, artificial intelligence (AI), data analytics, data engineering, and machine learning (ML) professionals.

- **Content Types**: 
  - **Latest Articles**: Insights into recent developments and practical applications in data science.
  - **Editors' Picks**: Curated articles highlighting must-read tips and trends.
  - **Deep Dives**: In-depth explorations of relevant topics in the data science field.

- **Notable Recent Articles**:
  - **Multimodal RAG**: A beginner's guide with Python code to process any file type using AI.
  - **Chat with Images**: Building a chat-like mode with the Llama 3.2-Vision model.
  - **Roles Explained**: Differences between data engineers and data scientists.
  - **Fantasy Football Agent**: A guide on creating a research agent using LangGraph and deploying it on AWS.
  - **Dynamic RAG**: Comparison between traditional and dynamic retrieval augmented generation.

- **Educational Resources**:
  - Articles aimed at transitioning into data science and within the discipline.
  - Guides on algorithmic considerations and management tips for new leaders in tech roles.

- **Community Engagement**: 
  - Options to contribute articles, sign up for newsletters, and follow the publication.

- **Navigation**: 
  - Various sections including Home, Latest Articles, Editors' Picks, About, and Archive, emphasizing ease of access to information.

- **Audience**: 
  - Targeted at professionals and aspiring individuals in the fields of data science, AI, data analytics, and ML.

These points encapsulate the essence of the 'Towards Data Science' website and its offerings.

In [76]:
# Gradio Interface
def main_interface(url, style):
    if url:
        summary = summarize(url, style)
        return summary
    else:
        return "Please provide a URL."

# Gradio app
interface = gr.Interface(
    fn=main_interface,
    inputs=[
        gr.Textbox(label="Enter URL for Summarization"),
        gr.Radio(["Key Points", "Detailed", "Q&A"], label="Choose Summary Style", value="Key Points"),
    ],
    outputs=[
        gr.Markdown(label="Summary")
    ],
    title="Website Summarizer",
    description="Paste a URL and choose a summary style."
)

# Launch the Gradio app
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7867

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
